<a href="https://colab.research.google.com/github/ykitaguchi77/GravCont_classification_colab/blob/master/Sort_Olympia_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Sort Oplympia dataset**

In [3]:
#GDriveをマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
parent_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset"

%cd $parent_path

/content/drive/MyDrive/Deep_learning/Olympia_dataset


In [16]:
import glob
import os
import shutil
import cv2


treated_id = os.listdir("treated_500px")
untreated_id = os.listdir("untreated")

# **横幅640pixelにresizeして別フォルダに保存**

In [21]:
# Create destination directories
basename_lists = [treated_id, untreated_id]
dst_dirs = ["treated_640px", "untreated_640px"]
for dir in dst_dirs:
    if os.path.exists(dir):
        shutil.rmtree(dir)
    os.makedirs(dir)

In [26]:
def resize_by_width(orig_dir, dst_dir, basename_list, width):
    path_list = glob.glob(f"{orig_dir}/*")
    for path in path_list:
        if os.path.basename(path) in basename_list:
            # Load the image
            img = cv2.imread(path)
            # Get the aspect ratio of the original image
            aspect_ratio = float(img.shape[0]) / img.shape[1]
            # Calculate the new height of the image
            new_height = int(640 * aspect_ratio)
            # Resize the image to 640xnew_height while maintaining aspect ratio
            resized_img = cv2.resize(img, (640, new_height), interpolation = cv2.INTER_LINEAR)
            # Save the resized image
            cv2.imwrite(f"{dst_dir}/{os.path.basename(path)}", resized_img)
            print(path)

In [27]:
for dst_dir, basename_list in zip(dst_dirs, basename_lists):
    resize_by_width("dataset", dir, basename_list, width=640)
    print(f"{dst_dir} done!!")

dataset/19.JPG
dataset/28.JPG
dataset/30.JPG
dataset/35.JPG
dataset/36.JPG
dataset/38.JPG
dataset/42.JPG
dataset/43.JPG
dataset/41.JPG
dataset/45.JPG
dataset/49.JPG
dataset/52.JPG
dataset/56.JPG
dataset/59.JPG
dataset/62.JPG
dataset/63.JPG
dataset/66.JPG
dataset/68.JPG
dataset/69.JPG
dataset/73.JPG
dataset/80.JPG
dataset/84.JPG
dataset/85.JPG
dataset/86.JPG
dataset/87.JPG
dataset/89.JPG
dataset/88.JPG
dataset/90.JPG
dataset/96.JPG
dataset/97.JPG
dataset/936.JPG
dataset/100.JPG
dataset/102.JPG
dataset/105.JPG
dataset/106.JPG
dataset/109.JPG
dataset/112.JPG
dataset/111.JPG
dataset/113.JPG
dataset/114.JPG
dataset/116.JPG
dataset/937.JPG
dataset/118.JPG
dataset/120.JPG
dataset/124.JPG
dataset/123.JPG
dataset/126.JPG
dataset/128.JPG
dataset/129.JPG
dataset/130.JPG
dataset/132.JPG
dataset/135.JPG
dataset/137.JPG
dataset/940.JPG
dataset/139.JPG
dataset/138.JPG
dataset/141.JPG
dataset/142.JPG
dataset/143.JPG
dataset/144.JPG
dataset/145.JPG
dataset/147.JPG
dataset/151.JPG
dataset/152.JPG
datase